# Analyser data fra kompensasjonsordningen med Python og Pandas

I denne notatboken brukes data om tildelinger av støtte fra Kompensasjonsordningen for næringslivet.

Dataene ligger som både .json og .csv i dette repositoriet, men inneholder ikke de siste tallene. Vedtak skjer løpende. Oppdaterte data legges ut hver time. Dette repo-et inneholder allerede en kopi av dataene fra mars i år. Ønsker du en oppdatert versjon kan du laste ned med f.eks. curl:


In [ ]:
# JSON
%curl https://data.brreg.no/kompensasjonsordning/api/tilskudd/json --output tilskudd.json

In [ ]:
# CSV
%curl https://data.brreg.no/kompensasjonsordning/api/tilskudd/csv --output tilskudd.csv

Steg 0: Installer Pandas

Dersom du ikke har gjort det allerede, installer Pandas (men husk den obligatoriske oppgraderingen av pip først). 

In [ ]:
%pip install --upgrade pip
%pip install pandas

Steg 1: Litt info om datasettet

Vi starter med å importere pandas, lese inn csv-filen, og få et inntrykk av hva den inneholder. Merk at CSV-fila bruker semikolon som skilletegn.




In [1]:
import pandas as pd

df = pd.read_csv('tilskudd.csv', sep=';')
df.head()


,tilskuddsnummer,tilskuddsmottakerOrganisasjonsnummer,tilskuddsmottakerNavn,kommunenummer,kommune,fylkesnummer,fylke,naeringskode1,naeringskode2,naeringskode3,tilskuddsbelop,tildelingsdato,tilskuddsperiode,sammenligningsperiode,sumFasteUungaaeligeKostanderITilskuddsperioden,omsetningITilskuddsperiode,omsetningISammenligningsperiode,justeringsfaktor,uavhengigMedsignererOrganisasjonsnummer,uavhengigMedsignererNavn
0,2021009917,913686454,AKTIV365 SANDSLI AS,4601,BERGEN,4600.0,Vestland,93.130,NaN,NaN,"125488,00",2021-03-08T00:00,november-desember2020,november-desember2019,"327907,57","766608,00","1394409,00",0.85,988782041,COLLEGIUM REVISJON AS
1,2021009916,988237612,GOLDER ASSOCIATES AS (NORWAY),3005,DRAMMEN,3000.0,Viken,71.129,NaN,NaN,"208706,41",2021-03-08T00:00,november-desember2020,november-desember2019,"554372,00","5147270,00","9239568,00",0.85,993606650,BDO AS
2,2021009915,967265403,ANITA GULBRANDSEN LØKEN,3020,NORDRE FOLLO,3000.0,Viken,49.320,NaN,NaN,"15008,28",2021-03-08T00:00,november-desember2020,november-desember2019,"34995,00","196945,00","397509,00",0.85,922829845,PARTNER REVISJON AS
3,2021009914,918718761,THAI AROMA MASSASJE AS,301,OSLO,300.0,Oslo,96.040,NaN,NaN,"7950,66",2021-03-08T00:00,november-desember2020,november-desember2019,"24905,00","81300,00","130200,00",0.85,913423585,MYHRERS RTB AS
4,2021009913,919965614,GLH LADE AS,5001,TRONDHEIM,5000.0,Trøndelag Trööndelage,47.599,NaN,NaN,"83134,37",2021-03-08T00:00,september-oktober2020,september-oktober2019,"287994,00","2328281,00","3962232,00",0.70,989271733,VEKTE AS


In [ ]:
Vi kan også få et uttrekk av helt tilfeldige rader, istedenfor bare de første, ved hjelp av sample(n) der n er antall rader:

In [3]:
df.sample(10)

,tilskuddsnummer,tilskuddsmottakerOrganisasjonsnummer,tilskuddsmottakerNavn,kommunenummer,kommune,fylkesnummer,fylke,naeringskode1,naeringskode2,naeringskode3,tilskuddsbelop,tildelingsdato,tilskuddsperiode,sammenligningsperiode,sumFasteUungaaeligeKostanderITilskuddsperioden,omsetningITilskuddsperiode,omsetningISammenligningsperiode,justeringsfaktor,uavhengigMedsignererOrganisasjonsnummer,uavhengigMedsignererNavn
4418,2021005084,916945825,BASTIANSEN CONSULTING AS,1103,STAVANGER,1100.0,Rogaland,70.220,NaN,NaN,"23279,59",2021-02-05T00:00,november-desember2020,november-desember2019,"28809,00","18500,00","375000,00",0.85,975378012,BOKHOLDERIET HINNA AS
2115,2021007553,918872434,CATERING INNLANDET AS,3411,RINGSAKER,3400.0,Innlandet,56.210,NaN,NaN,"9234,02",2021-02-22T00:00,november-desember2020,november-desember2019,"14855,00","95441,00","355203,00",0.85,985282013,DALA ØKONOMI AS
7047,2021002222,921334664,MINCHEV TAXI,3025,ASKER,3000.0,Viken,49.320,NaN,NaN,"12182,64",2021-01-25T00:00,september-oktober2020,september-oktober2019,"21659,00","23386,00","119034,00",0.70,913097300,ABE REGNSKAP AS
7540,2021001681,988225835,JAFFAR RAZA,3029,LØRENSKOG,3000.0,Viken,49.320,NaN,NaN,"7502,42",2021-01-22T00:00,november-desember2020,november-desember2019,"12682,00","157573,00","518292,00",0.85,937416873,ADMINISTRATIV REGNSKAPSSERVICE AS
6213,2021003126,993788406,FOKA ENTREPRENØR AS,3029,LØRENSKOG,3000.0,Viken,42.130,NaN,NaN,"118487,31",2021-01-27T00:00,september-oktober2020,september-oktober2019,"210813,00","596565,00","3027137,00",0.70,913191560,LØRENSKOG REGNSKAP AS
715,2021009126,917118167,JÆR TRE AS,1119,HÅ,1100.0,Rogaland,41.200,NaN,NaN,"42947,87",2021-03-03T00:00,september-oktober2020,september-oktober2019,"143112,00","2152529,00","3767865,00",0.70,993606650,BDO AS
6040,2021003317,815590872,BULJONGGRUPPEN AS,301,OSLO,300.0,Oslo,56.101,NaN,NaN,"483438,77",2021-01-28T00:00,november-desember2020,november-desember2019,"604971,00","311599,00","5204608,00",0.85,998485304,ASR ACCOUNTING AS
8882,2021000190,913570073,JULEBYGDA ENGROS & IMPORT AS,1108,SANDNES,1100.0,Rogaland,46.499,NaN,NaN,"166429,01",2021-01-18T00:00,november-desember2020,november-desember2019,"336345,00","2265864,00","5422503,00",0.85,982316588,RSM NORGE AS
1814,2021007906,998248175,LOTUS DRIFT AS,301,OSLO,300.0,Oslo,56.101,NaN,NaN,"63002,22",2021-02-23T00:00,september-oktober2020,september-oktober2019,"268427,00","1323634,00","1991321,00",0.70,998750881,MT REGNSKAP AS
3843,2021005699,982480582,AKTIV FYSIOTERAPI OG TRENING AS,3025,ASKER,3000.0,Viken,93.130,NaN,NaN,"37230,59",2021-02-10T00:00,september-oktober2020,september-oktober2019,"170715,00","493848,00","717335,00",0.70,998013593,LOS REGNSKAP AS


Vi ser at ... osv ...

Siden tabellen er så bred, kan det også være nyttig å bruke columns for å få en liste over alle kolonnenavnene:


In [5]:
df.columns

Index(['tilskuddsnummer', 'tilskuddsmottakerOrganisasjonsnummer',
       'tilskuddsmottakerNavn', 'kommunenummer', 'kommune', 'fylkesnummer',
       'fylke', 'naeringskode1', 'naeringskode2', 'naeringskode3',
       'tilskuddsbelop', 'tildelingsdato', 'tilskuddsperiode',
       'sammenligningsperiode',
       'sumFasteUungaaeligeKostanderITilskuddsperioden',
       'omsetningITilskuddsperiode', 'omsetningISammenligningsperiode',
       'justeringsfaktor', 'uavhengigMedsignererOrganisasjonsnummer',
       'uavhengigMedsignererNavn'],
      dtype='object')

Pandas har funksjon for å gi en rask oppsummering av noen statistiske egenskaper ved dataene i en dataframe, gjennom describe(). Da får vi blant annet antall verdier, median, minimum, maximum og gjennomsnitt (50 %-verdien):

In [2]:
df.describe()

,tilskuddsnummer,tilskuddsmottakerOrganisasjonsnummer,kommunenummer,fylkesnummer,naeringskode1,naeringskode2,naeringskode3,justeringsfaktor,uavhengigMedsignererOrganisasjonsnummer
count,9.048000e+03,9.048000e+03,9048.000000,9012.000000,9048.000000,244.000000,9.000000,9048.000000,9.048000e+03
mean,2.021005e+09,9.481143e+08,2662.386273,2651.364847,58.102011,54.737045,48.966889,0.789838,9.544368e+08
std,2.847589e+03,4.406056e+07,1772.681531,1772.489596,17.751790,22.596203,29.426245,0.073522,4.161825e+07
min,2.021000e+09,8.116188e+08,301.000000,300.000000,0.000000,0.000000,10.710000,0.700000,8.121884e+08
25%,2.021003e+09,9.168709e+08,1103.000000,1100.000000,49.320000,47.285250,15.200000,0.700000,9.206667e+08
50%,2.021005e+09,9.579039e+08,3024.000000,3000.000000,56.101000,55.202000,55.202000,0.850000,9.744623e+08
75%,2.021007e+09,9.880344e+08,4601.000000,4600.000000,70.220000,70.100000,78.200000,0.850000,9.875649e+08
max,2.021010e+09,9.996654e+08,5444.000000,5400.000000,96.090000,96.020000,79.909000,0.850000,9.996475e+08


Men her ser vi at det som det er mest interessant å få frem verdier på er ikke med, for eksempel beløpene for tilskudd og omsetningstall. Det skyldes at disse verdiene leses inn som tekst istedenfor tall, pga komma som desimaltegn. Til gjengjeld får vi masse statistiske verdier for numre som er identifikatorer og ikke tall, f.eks. organisasjonsnummer, næringskode, kommunenummer. For å bekrefte det kan vi se nærmere på en enkelt kolonne, og se hvilken type (dtype) den har:


In [8]:
df['tilskuddsbelop']


0       125488,00
1       208706,41
2        15008,28
3         7950,66
4        83134,37
          ...    
9043     20367,53
9044     56709,39
9045      9844,00
9046     11624,90
9047    156339,75
Name: tilskuddsbelop, Length: 9048, dtype: object

In [9]:
df['tilskuddsnummer']

0       2021009917
1       2021009916
2       2021009915
3       2021009914
4       2021009913
           ...    
9043    2021000008
9044    2021000007
9045    2021000006
9046    2021000005
9047    2021000004
Name: tilskuddsnummer, Length: 9048, dtype: int64

Beløp er altså den generelle "object"-typen mens identifikatoren tilskuddsnummer er tall (int64)

Årsaken til at beløp blir tekst er at desimalskilletegnet i Pandas er punktum, mens vi bruker komma i Norge. Så for ved å presisere dette når vi leser inn Excel-fila igjen kan vi få rettet opp noen av datatypene:

In [11]:
df = pd.read_csv('tilskudd.csv', sep=';', decimal=',')


In [12]:
df.describe()

,tilskuddsnummer,tilskuddsmottakerOrganisasjonsnummer,kommunenummer,fylkesnummer,tilskuddsbelop,sumFasteUungaaeligeKostanderITilskuddsperioden,omsetningITilskuddsperiode,omsetningISammenligningsperiode,uavhengigMedsignererOrganisasjonsnummer
count,9.048000e+03,9.048000e+03,9048.000000,9012.000000,9.048000e+03,9.048000e+03,9.048000e+03,9.048000e+03,9.048000e+03
mean,2.021005e+09,9.481143e+08,2662.386273,2651.364847,1.910668e+05,4.228766e+05,1.433093e+06,1.424610e+07,9.544368e+08
std,2.847589e+03,4.406056e+07,1772.681531,1772.489596,2.037838e+06,4.467720e+06,8.715873e+06,1.013714e+09,4.161825e+07
min,2.021000e+09,8.116188e+08,301.000000,300.000000,5.000340e+03,6.410000e+03,0.000000e+00,6.000000e+03,8.121884e+08
25%,2.021003e+09,9.168709e+08,1103.000000,1100.000000,1.718204e+04,4.087875e+04,1.117474e+05,3.516660e+05,9.206667e+08
50%,2.021005e+09,9.579039e+08,3024.000000,3000.000000,4.043306e+04,9.759950e+04,3.093570e+05,9.054335e+05,9.744623e+08
75%,2.021007e+09,9.880344e+08,4601.000000,4600.000000,1.023590e+05,2.405640e+05,9.022215e+05,2.425666e+06,9.875649e+08
max,2.021010e+09,9.996654e+08,5444.000000,5400.000000,8.188874e+07,1.918143e+08,4.146793e+08,9.640463e+10,9.996475e+08


Nå ser vi at beløpene er blitt tall, og til gjengjeld er næringskodene ikke lenger tall. Det hjalp litt. I tillegg kan vi endre datatypen til de ulike numrene.
[TODO]

In [13]:
df['tilskuddsbelop']

0       125488.00
1       208706.41
2        15008.28
3         7950.66
4        83134.37
          ...    
9043     20367.53
9044     56709.39
9045      9844.00
9046     11624.90
9047    156339.75
Name: tilskuddsbelop, Length: 9048, dtype: float64

In [14]:
df['tilskuddsbelop'].describe().round(2)

count        9048.00
mean       191066.80
std       2037838.13
min          5000.34
25%         17182.04
50%         40433.06
75%        102358.97
max      81888735.97
Name: tilskuddsbelop, dtype: float64

In [ ]:
df[df['tilskuddsbelop']==5000.34]

In [ ]:
%pip install requests

In [ ]:
import requests

Hent mer info om selskapet som fikk minst støtte, gjennom API-et for åpne data fra Enhetsregisteret. Først lagrer vi URL-en til API-et.

In [ ]:
ERAPIURL = 'https://data.brreg.no/enhetsregisteret/api/enheter/'
r = requests.get(ERAPIURL + '982575044')
r.text

In [ ]:
REGNSKAPAPIURL = 'http://34.98.91.231/regnskap/'
r_regnskap = requests.get(REGNSKAPAPIURL + '982575044')
r_regnskap.text

## Næringskoder
Det er tre kolonner med det vi kaller "næringskoder". Det er en kode Brønnøysundregistrene fastsetter basert på hva som er bedriftenes formål. Næringskoden har til hensikt å gjøre det mulig for SSB å lage statistikk. I andre land er det bedriftene selv som velger næringskode. Brønnøysundregistrene har tatt i bruk maskinlæring for å hjelpe saksbehandlerne med å fastsette næringskode, og de som er interesserte kan se mer om dette her:
1) demo-grensesnitt for å få forslag til næringskode
2) github med kode

Det er SSB som forvalter næringskodene, og på deres side [Standard for næringsgruppering (SN)](https://www.ssb.no/klass/klassifikasjoner/6) kan dere lese mer om kodene. Her kan dere også laste ned en fil med alle næringskode og beskrivelse av dem. Den skal vi bruke videre. Den ligger i repoet med tittelen nace.csv. Hvis dere vil laste den ned selv, er URL-en https://data.ssb.no/api/klass/v1//versions/30.csv?language=nb 

Merk valg for separator og encoding når vi leser inn fila. Og som dere ser av utvalget dere får opp med "sample" inneholder den nye dataframen norske navn på kodene i kolonnen 'name'.

In [22]:
df_nace = pd.read_csv('nace.csv', sep=';', encoding='iso8859-1')
df_nace.sample(5)

,code,parentCode,level,name,shortName,notes,validFrom,validTo
1609,86.9,86,3,Andre helsetjenester,Andre helsetjenester,NaN,NaN,NaN
915,46.48,46.4,4,"Engroshandel med klokker, ur, gull- og sølvvarer",Engrosh. klokker/ur/gull-/sølvvarer,NaN,NaN,NaN
985,46.630,46.63,5,Engroshandel med maskiner og utstyr til bergve...,Engros mask bygg/anl./oljeutv. mv.,NaN,NaN,NaN
1261,59.140,59.14,5,Filmframvisning,Filmframvisning,Inkluderer: Omfatter framvisning av film eller...,NaN,NaN
605,28.11,28.1,4,"Produksjon av motorer og turbiner, unntatt mot...",Skipsmotor-/motor- og turbinprod. el.,Inkluderer: Omfatter produksjon av forbrenning...,NaN,NaN


In [25]:
df_nace['code']

0         01
1       01.1
2       01.2
3       01.3
4       01.4
        ... 
1806       Q
1807       R
1808       S
1809       T
1810       U
Name: code, Length: 1811, dtype: object

In [ ]:
Det er den første kolonnen, code, som inneholder alle kodene. I vår opprinnelige dataframe med alle tilskuddene er det tre ulike næringskoder, men den første representerer hovedaktiviteten, så vi bryr oss bare om den. For å gjøre det enklere for oss, lager vi en ny dataframe, som sier litt om innholdet, og endrer samtidig navn på den kolonnene med næringskoder vi er interessert i til samme som i df_nace. På den måten kan vi enklere koble informasjonen mellom de to:

In [15]:
df_tilskudd = df.rename(columns={'naeringskode1':'code'})

In [ ]:
Nå som vi har to dataframes, som begge har en kolonne "code", kan vi bruke "merge"-funksjonen til å slå innholdet sammen. 

Situasjonen før er at vi bare har en kode av typen '93.130' som vi ikke vet hva betyr:

In [16]:
df_tilskudd.head()

,tilskuddsnummer,tilskuddsmottakerOrganisasjonsnummer,tilskuddsmottakerNavn,kommunenummer,kommune,fylkesnummer,fylke,code,naeringskode2,naeringskode3,tilskuddsbelop,tildelingsdato,tilskuddsperiode,sammenligningsperiode,sumFasteUungaaeligeKostanderITilskuddsperioden,omsetningITilskuddsperiode,omsetningISammenligningsperiode,justeringsfaktor,uavhengigMedsignererOrganisasjonsnummer,uavhengigMedsignererNavn
0,2021009917,913686454,AKTIV365 SANDSLI AS,4601,BERGEN,4600.0,Vestland,93.130,NaN,NaN,125488.00,2021-03-08T00:00,november-desember2020,november-desember2019,327907.57,766608.0,1394409.0,0.85,988782041,COLLEGIUM REVISJON AS
1,2021009916,988237612,GOLDER ASSOCIATES AS (NORWAY),3005,DRAMMEN,3000.0,Viken,71.129,NaN,NaN,208706.41,2021-03-08T00:00,november-desember2020,november-desember2019,554372.00,5147270.0,9239568.0,0.85,993606650,BDO AS
2,2021009915,967265403,ANITA GULBRANDSEN LØKEN,3020,NORDRE FOLLO,3000.0,Viken,49.320,NaN,NaN,15008.28,2021-03-08T00:00,november-desember2020,november-desember2019,34995.00,196945.0,397509.0,0.85,922829845,PARTNER REVISJON AS
3,2021009914,918718761,THAI AROMA MASSASJE AS,301,OSLO,300.0,Oslo,96.040,NaN,NaN,7950.66,2021-03-08T00:00,november-desember2020,november-desember2019,24905.00,81300.0,130200.0,0.85,913423585,MYHRERS RTB AS
4,2021009913,919965614,GLH LADE AS,5001,TRONDHEIM,5000.0,Trøndelag Trööndelage,47.599,NaN,NaN,83134.37,2021-03-08T00:00,september-oktober2020,september-oktober2019,287994.00,2328281.0,3962232.0,0.70,989271733,VEKTE AS


In [ ]:
Så kombinerer vi de to dataframene, og dermed får vi tekster for næringskodene i kolonnen 'name':

In [21]:
df_extended = pd.merge(df_tilskudd, df_nace)
df_extended.sample(5)

,tilskuddsnummer,tilskuddsmottakerOrganisasjonsnummer,tilskuddsmottakerNavn,kommunenummer,kommune,fylkesnummer,fylke,code,naeringskode2,naeringskode3,...,justeringsfaktor,uavhengigMedsignererOrganisasjonsnummer,uavhengigMedsignererNavn,parentCode,level,name,shortName,notes,validFrom,validTo
6544,2021007650,899014332,BERGEN RESTAURANTKANTINE AS,4601,BERGEN,4600.0,Vestland,56.290,NaN,NaN,...,0.70,963048386,OLSTAD REVISJON AS,56.29,5,Kantiner drevet som selvstendig virksomhet,"Kantinedrift, selvst. virksomhet",Inkluderer: Omfatter underleverandører av mat ...,NaN,NaN
7050,2021007361,987919043,RUDI KULTUR AS,3438,SØR-FRON,3400.0,Innlandet,90.020,NaN,NaN,...,0.85,967673374,FRON REGNSKAPSKONTOR SA,90.02,5,Tjenester tilknyttet underholdningsvirksomhet,Tjen. tilkn. underholdningsvirks.,Inkluderer: Omfatter tjenester i forbindelse m...,NaN,NaN
8435,2021006340,979194803,BRENNERIVEIEN JAZZHUS AS,301,OSLO,300.0,Oslo,90.040,NaN,NaN,...,0.70,912129802,STATEMENT AS,90.04,5,Drift av lokaler tilknyttet kunstnerisk virkso...,Drift lokaler tilkn. kunst.virks.,Inkluderer: Omfatter drift av konsert- og teat...,NaN,NaN
3983,2021001090,995437635,H. L. BAKKEKRO AS,301,OSLO,300.0,Oslo,56.101,NaN,NaN,...,0.85,982903378,VIC CONSULT AS,56.10,5,Drift av restauranter og kafeer,Drift av restauranter og kafeer,Inkluderer også: Omfatter også togservering Ek...,NaN,NaN
4421,2021005399,912009181,AS GLEA,1856,RØST,1800.0,Nordland,10.201,NaN,NaN,...,0.70,950485922,ØKORÅD NARVIK AS,10.20,5,"Produksjon av saltfisk, tørrfisk og klippfisk","Salt-, tørr- og klippfiskproduksjon",NaN,NaN,NaN
